In [1]:
%reload_ext autoreload
%autoreload
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [15]:
%autoreload now
from source.script import main_settings

params, options = main_settings("meerkat-src-100-config.yml")

[2023-05-26 10:27:06,647 | INFO | source.script] Fetching directory paths
[2023-05-26 10:27:06,660 | INFO | source.script] Creating new path data
[2023-05-26 10:27:06,672 | INFO | source.script] Completed and returning paths
[2023-05-26 10:27:06,673 | INFO | source.script] Creating directory paths
[2023-05-26 10:27:06,673 | INFO | source.script] Making directories
[2023-05-26 10:27:06,676 | INFO | source.script] Path data saved to file
[2023-05-26 10:27:06,677 | INFO | source.script] Setting thread count to 24
[2023-05-26 10:27:06,678 | INFO | source.script] Setting random state seed to 666
[2023-05-26 10:27:06,679 | INFO | source.script] Matplotlib config directory unchanged
[2023-05-26 10:27:06,679 | INFO | source.script] Creating new parameter data
[2023-05-26 10:27:06,688 | INFO | source.script] Completed and returning parameters
[2023-05-26 10:27:06,688 | INFO | source.script] Simulation setup complete


In [24]:
%autoreload now
from source.plotting import setup_plotting, create_amplitude_and_phase_signal_plot
from source.data import (
    create_empty_measurement_set, 
    create_gains_signal, 
    create_skymodels, 
    create_visibilities
)

# Setup plotting
setup_plotting(options["plots"], params)

# Create measurement set
create_empty_measurement_set(options["ms"], params)

# Create true gains signal
create_gains_signal(options["gains"], params)

# Create sky models
create_skymodels(options["vis"], params)

# Create visibilities
create_visibilities(options["vis"], params)

# Plot amplitude and signal
create_amplitude_and_phase_signal_plot([0, 1, 2], params, show=True)

[2023-05-26 10:31:52,662 | INFO | source.plotting] Retrieving plotting options
[2023-05-26 10:31:52,664 | INFO | source.plotting] Updating parameter data
[2023-05-26 10:31:52,693 | INFO | source.data] Retrieving measurement set options
[2023-05-26 10:31:52,695 | INFO | source.data] Name/Path: `/net/sinatra/vault2-tina/welman/meerkat-src-100/meerkat.ms`
[2023-05-26 10:31:52,695 | INFO | source.data] Telescope: meerkat
[2023-05-26 10:31:52,696 | INFO | source.data] Right Ascension: 11h50m15s
[2023-05-26 10:31:52,697 | INFO | source.data] Declination: -30d27m43s
[2023-05-26 10:31:52,698 | INFO | source.data] Synthesis-Time: 2 hr
[2023-05-26 10:31:52,698 | INFO | source.data] Integration-Time: 10 s
[2023-05-26 10:31:52,699 | INFO | source.data] Initial Frequency: 1GHz
[2023-05-26 10:31:52,700 | WARNING | source.other] The following `1` files/folders already exist:
[2023-05-26 10:31:52,701 | WARNING | source.other] Delete? `/net/sinatra/vault2-tina/welman/meerkat-src-100/meerkat.ms`


Do you wish to replace it? (y/n) n


[2023-05-26 10:31:55,671 | INFO | source.data] Keep original measurement set at `meerkat.ms`
[2023-05-26 10:31:55,673 | INFO | source.data] Updating path and parameter data
[2023-05-26 10:31:55,694 | INFO | source.data] Setting seed to 666
[2023-05-26 10:31:55,695 | INFO | source.data] Updating path data
[2023-05-26 10:31:55,696 | INFO | source.data] Retrieving information from `/net/sinatra/vault2-tina/welman/meerkat-src-100/meerkat.ms`
[2023-05-26 10:31:55,760 | INFO | source.data] Calculating dimensions and axes
[2023-05-26 10:31:55,843 | INFO | source.data] Generating full-complex gains
[2023-05-26 10:31:55,961 | WARNING | source.other] The following `1` files/folders already exist:
[2023-05-26 10:31:55,963 | WARNING | source.other] Delete? `/net/sinatra/vault2-tina/welman/meerkat-src-100/gains/true/true-gains.npz`


Do you wish to replace it? (y/n) y


[2023-05-26 10:31:57,248 | INFO | source.data] Codex shape: (N_ant, N_time, N_chan, N_dir, N_corr) = (720, 64, 1, 1, 1)
[2023-05-26 10:31:57,250 | INFO | source.data] Saving gains to file
[2023-05-26 10:31:57,263 | INFO | source.data] New simulated gains at `/net/sinatra/vault2-tina/welman/meerkat-src-100/gains/true/true-gains.npz`
[2023-05-26 10:31:57,268 | INFO | source.data] Setting seed to 666
[2023-05-26 10:31:57,270 | INFO | source.data] Updating path data
[2023-05-26 10:31:57,284 | INFO | source.data] Fetching skymodel information
[2023-05-26 10:31:57,285 | INFO | source.data] Updating parameter information
[2023-05-26 10:31:57,689 | INFO | source.data] Determining flux and layout information
[2023-05-26 10:31:57,726 | WARNING | source.other] The following `4` files/folders already exist:
[2023-05-26 10:31:57,727 | WARNING | source.other] Delete? `/net/sinatra/vault2-tina/welman/meerkat-src-100/fluxes/true/model-100mp.npz`
[2023-05-26 10:31:57,728 | WARNING | source.other] Delet

...

[2023-05-26 10:31:59,368 | INFO | source.data] No deletion done
[2023-05-26 10:31:59,373 | INFO | source.data] Setting seed to 666
[2023-05-26 10:31:59,375 | INFO | source.data] Fetching visibility information
[2023-05-26 10:31:59,659 | INFO | source.data] Load the true-gains data
[2023-05-26 10:31:59,669 | INFO | source.data] Generating visibilities
[2023-05-26 10:32:00,304 | INFO | source.data] Saving visibilities to `/net/sinatra/vault2-tina/welman/meerkat-src-100/meerkat.ms`
[2023-05-26 10:32:00,315 | WARNING | source.data] Column `NOISE_100MP` exists


Replace `NOISE_100MP`? (y/n/a) n


[2023-05-26 10:32:01,320 | INFO | source.data] Stopping visibility generation
[2023-05-26 10:32:01,337 | INFO | source.plotting] Setting seed to 666
[2023-05-26 10:32:01,339 | WARNING | source.other] The following `1` files/folders already exist:
[2023-05-26 10:32:01,340 | WARNING | source.other] Delete? `meerkat-src-100/plots/ch5-sim-gains.png`


Do you wish to replace it? (y/n) n


[2023-05-26 10:32:02,300 | INFO | source.plotting] No plotting done


In [51]:
# %%capture kalcal_diag_cal_run
%autoreload now
from source.algorithms import run_kalcal_diag_calibration

run_kalcal_diag_calibration(options["kalcal-diag"], params, progress=True, check_mse=True)

Runs: 0/? [00:00<?, ?it/s]

[2023-05-25 18:49:27,562 | INFO | source.algorithms] Setting seed to 666
[2023-05-25 18:49:27,563 | INFO | source.algorithms] Fetching kalcal-diag calibration run options
[2023-05-25 18:49:27,564 | INFO | source.algorithms] Calculated line search process noise parameters
[2023-05-25 18:49:27,565 | INFO | source.algorithms] Updating path data
[2023-05-25 18:49:28,101 | WARNING | source.algorithms] May take long to start. `numba` is compiling the functions.
[2023-05-25 18:49:28,102 | INFO | source.algorithms] Running line-search on 240 points (960 runs)
[2023-05-25 18:49:28,103 | INFO | source.algorithms] Using interval [1e-5, 1e0].
[2023-05-25 18:49:36,704 | INFO | source.algorithms] kalcal-diag on 100MP with `sigma_f=1.000e-05`, 8.59s taken, with filter-MSE=0.0759, smoother-MSE=0.0705
[2023-05-25 18:49:37,069 | INFO | source.algorithms] kalcal-diag on 100MP with `sigma_f=1.049e-05`, 0.348s taken, with filter-MSE=0.0759, smoother-MSE=0.0705
[2023-05-25 18:49:37,424 | INFO | source.algor

[2023-05-25 18:49:53,628 | INFO | source.algorithms] kalcal-diag on 100MP with `sigma_f=1.010e-04`, 0.324s taken, with filter-MSE=0.0759, smoother-MSE=0.0705
[2023-05-25 18:49:53,970 | INFO | source.algorithms] kalcal-diag on 100MP with `sigma_f=1.060e-04`, 0.327s taken, with filter-MSE=0.0759, smoother-MSE=0.0705
[2023-05-25 18:49:54,325 | INFO | source.algorithms] kalcal-diag on 100MP with `sigma_f=1.112e-04`, 0.335s taken, with filter-MSE=0.0759, smoother-MSE=0.0705
[2023-05-25 18:49:54,679 | INFO | source.algorithms] kalcal-diag on 100MP with `sigma_f=1.167e-04`, 0.338s taken, with filter-MSE=0.0759, smoother-MSE=0.0705
[2023-05-25 18:49:55,028 | INFO | source.algorithms] kalcal-diag on 100MP with `sigma_f=1.224e-04`, 0.332s taken, with filter-MSE=0.0759, smoother-MSE=0.0705
[2023-05-25 18:49:55,383 | INFO | source.algorithms] kalcal-diag on 100MP with `sigma_f=1.285e-04`, 0.34s taken, with filter-MSE=0.0759, smoother-MSE=0.0705
[2023-05-25 18:49:55,728 | INFO | source.algorithms] 

[2023-05-25 18:50:15,656 | INFO | source.algorithms] kalcal-diag on 100MP with `sigma_f=1.236e-03`, 0.326s taken, with filter-MSE=0.0754, smoother-MSE=0.0696
[2023-05-25 18:50:15,991 | INFO | source.algorithms] kalcal-diag on 100MP with `sigma_f=1.297e-03`, 0.32s taken, with filter-MSE=0.0753, smoother-MSE=0.0695
[2023-05-25 18:50:16,326 | INFO | source.algorithms] kalcal-diag on 100MP with `sigma_f=1.361e-03`, 0.319s taken, with filter-MSE=0.0753, smoother-MSE=0.0695
[2023-05-25 18:50:16,664 | INFO | source.algorithms] kalcal-diag on 100MP with `sigma_f=1.428e-03`, 0.323s taken, with filter-MSE=0.0752, smoother-MSE=0.0694
[2023-05-25 18:50:17,000 | INFO | source.algorithms] kalcal-diag on 100MP with `sigma_f=1.499e-03`, 0.32s taken, with filter-MSE=0.0751, smoother-MSE=0.0693
[2023-05-25 18:50:17,333 | INFO | source.algorithms] kalcal-diag on 100MP with `sigma_f=1.573e-03`, 0.318s taken, with filter-MSE=0.075, smoother-MSE=0.0691
[2023-05-25 18:50:17,667 | INFO | source.algorithms] ka

[2023-05-25 18:50:37,286 | INFO | source.algorithms] kalcal-diag on 100MP with `sigma_f=1.588e-02`, 0.331s taken, with filter-MSE=0.0576, smoother-MSE=0.0485
[2023-05-25 18:50:37,635 | INFO | source.algorithms] kalcal-diag on 100MP with `sigma_f=1.666e-02`, 0.333s taken, with filter-MSE=0.057, smoother-MSE=0.0479
[2023-05-25 18:50:37,990 | INFO | source.algorithms] kalcal-diag on 100MP with `sigma_f=1.749e-02`, 0.341s taken, with filter-MSE=0.0564, smoother-MSE=0.0473
[2023-05-25 18:50:38,349 | INFO | source.algorithms] kalcal-diag on 100MP with `sigma_f=1.835e-02`, 0.343s taken, with filter-MSE=0.0558, smoother-MSE=0.0467
[2023-05-25 18:50:38,703 | INFO | source.algorithms] kalcal-diag on 100MP with `sigma_f=1.925e-02`, 0.338s taken, with filter-MSE=0.0552, smoother-MSE=0.046
[2023-05-25 18:50:39,059 | INFO | source.algorithms] kalcal-diag on 100MP with `sigma_f=2.020e-02`, 0.34s taken, with filter-MSE=0.0546, smoother-MSE=0.0454
[2023-05-25 18:50:39,410 | INFO | source.algorithms] ka

...

[2023-05-25 18:54:14,543 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=2.060e-05`, 0.332s taken, with filter-MSE=0.0779, smoother-MSE=0.0729
[2023-05-25 18:54:14,884 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=2.161e-05`, 0.325s taken, with filter-MSE=0.0779, smoother-MSE=0.0729
[2023-05-25 18:54:15,235 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=2.268e-05`, 0.332s taken, with filter-MSE=0.0779, smoother-MSE=0.0729
[2023-05-25 18:54:15,571 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=2.380e-05`, 0.32s taken, with filter-MSE=0.0779, smoother-MSE=0.0729
[2023-05-25 18:54:16,066 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=2.497e-05`, 0.427s taken, with filter-MSE=0.0779, smoother-MSE=0.0729
[2023-05-25 18:54:16,732 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=2.621e-05`, 0.598s taken, with filter-MSE=0.0779, smoother-MSE=0.0729
[2023-05-25 18:54:17,394 | INFO | source.algorithms] kalcal

[2023-05-25 18:54:34,423 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=2.646e-04`, 0.322s taken, with filter-MSE=0.0779, smoother-MSE=0.0729
[2023-05-25 18:54:34,769 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=2.777e-04`, 0.329s taken, with filter-MSE=0.0779, smoother-MSE=0.0729
[2023-05-25 18:54:35,113 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=2.914e-04`, 0.329s taken, with filter-MSE=0.0779, smoother-MSE=0.0729
[2023-05-25 18:54:35,462 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=3.057e-04`, 0.333s taken, with filter-MSE=0.0779, smoother-MSE=0.0729
[2023-05-25 18:54:35,820 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=3.208e-04`, 0.343s taken, with filter-MSE=0.0779, smoother-MSE=0.0729
[2023-05-25 18:54:36,155 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=3.367e-04`, 0.321s taken, with filter-MSE=0.0779, smoother-MSE=0.0728
[2023-05-25 18:54:36,509 | INFO | source.algorithms] kalca

[2023-05-25 18:54:54,760 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=3.399e-03`, 0.329s taken, with filter-MSE=0.0757, smoother-MSE=0.0695
[2023-05-25 18:54:55,237 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=3.567e-03`, 0.413s taken, with filter-MSE=0.0756, smoother-MSE=0.0693
[2023-05-25 18:54:55,774 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=3.743e-03`, 0.522s taken, with filter-MSE=0.0754, smoother-MSE=0.069
[2023-05-25 18:54:56,107 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=3.928e-03`, 0.316s taken, with filter-MSE=0.0752, smoother-MSE=0.0687
[2023-05-25 18:54:56,453 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=4.122e-03`, 0.33s taken, with filter-MSE=0.075, smoother-MSE=0.0684
[2023-05-25 18:54:56,791 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=4.325e-03`, 0.323s taken, with filter-MSE=0.0747, smoother-MSE=0.0681
[2023-05-25 18:54:57,135 | INFO | source.algorithms] kalcal-d

[2023-05-25 18:55:13,826 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=4.367e-02`, 0.466s taken, with filter-MSE=0.056, smoother-MSE=0.0466
[2023-05-25 18:55:14,428 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=4.582e-02`, 0.532s taken, with filter-MSE=0.0556, smoother-MSE=0.0462
[2023-05-25 18:55:15,030 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=4.808e-02`, 0.539s taken, with filter-MSE=0.0551, smoother-MSE=0.0457
[2023-05-25 18:55:15,636 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=5.046e-02`, 0.54s taken, with filter-MSE=0.0546, smoother-MSE=0.0453
[2023-05-25 18:55:16,052 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=5.295e-02`, 0.394s taken, with filter-MSE=0.0542, smoother-MSE=0.0449
[2023-05-25 18:55:16,410 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=5.556e-02`, 0.341s taken, with filter-MSE=0.0537, smoother-MSE=0.0444
[2023-05-25 18:55:16,748 | INFO | source.algorithms] kalcal-

[2023-05-25 18:55:34,358 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=5.610e-01`, 0.337s taken, with filter-MSE=0.0444, smoother-MSE=0.0385
[2023-05-25 18:55:34,715 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=5.887e-01`, 0.341s taken, with filter-MSE=0.0447, smoother-MSE=0.0389
[2023-05-25 18:55:35,056 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=6.177e-01`, 0.325s taken, with filter-MSE=0.0449, smoother-MSE=0.0392
[2023-05-25 18:55:35,415 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=6.482e-01`, 0.34s taken, with filter-MSE=0.0452, smoother-MSE=0.0396
[2023-05-25 18:55:35,764 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=6.802e-01`, 0.334s taken, with filter-MSE=0.0455, smoother-MSE=0.04
[2023-05-25 18:55:36,114 | INFO | source.algorithms] kalcal-diag on 25MP with `sigma_f=7.138e-01`, 0.335s taken, with filter-MSE=0.0458, smoother-MSE=0.0404
[2023-05-25 18:55:36,460 | INFO | source.algorithms] kalcal-d

In [ ]:
%autoreload now
from source.algorithms import run_kalcal_full_calibration

run_kalcal_full_calibration(options["kalcal-full"], params, progress=True, check_mse=True)

In [ ]:
%%capture quartical_calibration
%autoreload now
from source.algorithms import run_quartical_calibration

run_quartical_calibration(options["quartical"], params, progress=True, check_mse=True)

[2023-05-25 23:42:38,727 | INFO | source.algorithms] Setting seed to 666
[2023-05-25 23:42:38,728 | INFO | source.algorithms] Running QuartiCal setup
[2023-05-25 23:42:38,730 | INFO | source.algorithms] Updating path data
[2023-05-25 23:42:38,750 | WARNING | source.other] The following `1` files/folders already exist:
[2023-05-25 23:42:38,751 | WARNING | source.other] Delete? `meerkat-src-100/config/quartical.yml`


Do you wish to replace it? (y/n) y


[2023-05-25 23:42:44,495 | INFO | source.algorithms] Fetching QuartiCal option information
[2023-05-25 23:42:44,542 | INFO | source.algorithms] Creating QuartiCal config
[2023-05-25 23:42:44,553 | INFO | source.algorithms] Updating parameter data
[2023-05-25 23:42:44,582 | INFO | source.algorithms] Retrieved line search t-ints parameters
[2023-05-25 23:42:44,583 | INFO | source.algorithms] Updating path data
[2023-05-25 23:42:44,601 | INFO | source.algorithms] Running line-search on 240 points (960 runs)
[2023-05-25 23:42:50,152 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=1`, 5.51s taken, with MSE=0.255
[2023-05-25 23:42:55,172 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=2`, 4.98s taken, with MSE=0.116
[2023-05-25 23:43:00,618 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=3`, 5.4s taken, with MSE=0.0748
[2023-05-25 23:43:05,488 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=4`, 4.83s taken, with MSE=0.0551
[2023-05-25 23:43:10,308 

[2023-05-25 23:48:13,920 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=67`, 4.85s taken, with MSE=0.03
[2023-05-25 23:48:18,707 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=68`, 4.74s taken, with MSE=0.03
[2023-05-25 23:48:23,561 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=69`, 4.81s taken, with MSE=0.0302
[2023-05-25 23:48:28,434 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=70`, 4.83s taken, with MSE=0.0305
[2023-05-25 23:48:33,293 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=71`, 4.82s taken, with MSE=0.0308
[2023-05-25 23:48:38,166 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=72`, 4.83s taken, with MSE=0.031
[2023-05-25 23:48:43,058 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=73`, 4.85s taken, with MSE=0.031
[2023-05-25 23:48:47,986 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=74`, 4.88s taken, with MSE=0.031
[2023-05-25 23:48:52,823 | INFO | source.algorithms] QuartiCal 

[2023-05-25 23:53:56,157 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=137`, 4.97s taken, with MSE=0.0375
[2023-05-25 23:54:01,015 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=138`, 4.81s taken, with MSE=0.0377
[2023-05-25 23:54:05,798 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=139`, 4.75s taken, with MSE=0.0378
[2023-05-25 23:54:10,664 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=140`, 4.82s taken, with MSE=0.0379
[2023-05-25 23:54:15,606 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=141`, 4.9s taken, with MSE=0.0381
[2023-05-25 23:54:20,447 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=142`, 4.8s taken, with MSE=0.0383
[2023-05-25 23:54:25,373 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=143`, 4.88s taken, with MSE=0.0386
[2023-05-25 23:54:30,243 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=144`, 4.82s taken, with MSE=0.0386
[2023-05-25 23:54:35,113 | INFO | source.algorithm

[2023-05-25 23:59:34,065 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=206`, 4.74s taken, with MSE=0.0422
[2023-05-25 23:59:38,966 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=207`, 4.86s taken, with MSE=0.0423
[2023-05-25 23:59:43,882 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=208`, 4.87s taken, with MSE=0.0423
[2023-05-25 23:59:48,676 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=209`, 4.75s taken, with MSE=0.0424
[2023-05-25 23:59:53,733 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=210`, 5.01s taken, with MSE=0.0424
[2023-05-25 23:59:58,573 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=211`, 4.8s taken, with MSE=0.0425
[2023-05-26 00:00:03,491 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=212`, 4.88s taken, with MSE=0.0426
[2023-05-26 00:00:08,372 | INFO | source.algorithms] QuartiCal on 100MP with `t-int=213`, 4.84s taken, with MSE=0.0427
[2023-05-26 00:00:13,229 | INFO | source.algorith

In [ ]:
%%capture quartical_imaging
%autoreload now
from source.imaging import run_quartical_wsclean_imaging
import numpy as np

run_quartical_wsclean_imaging(params)

In [26]:
%autoreload now
from source.imaging import run_single_wsclean_imaging

paths = params["paths"]
gains_path = paths["gains"]["true"]
fits_name = "true"

run_single_wsclean_imaging(gains_path, fits_name, params)

[2023-05-26 10:32:35,775 | INFO | source.imaging] Updating path data
[2023-05-26 10:32:35,790 | INFO | source.imaging] Fetching imaging parameters



WSClean version 2.10.1 (2020-07-20)
This software package is released under the GPL version 3.
Author: André Offringa (offringa@gmail.com).

=== IMAGING TABLE ===
       # Pol Ch JG ²G In Freq(MHz)
| Independent group:
+-+-J- 0 XX   0  0  0  0  999-1000 (1)

 == Constructing PSF ==
Precalculating weights for Briggs'(0) weighting...
Opening /net/sinatra/vault2-tina/welman/meerkat-src-100/meerkat.ms, spw 0 with contiguous MS reader.
Mapping measurement set rows... DONE (0-1451520; 1451520 rows)

Detected 503.8 GB of system memory, usage limited to 100 GB (frac=100%, limit=100GB)
Opening /net/sinatra/vault2-tina/welman/meerkat-src-100/meerkat.ms, spw 0 with contiguous MS reader.
Mapping measurement set rows... DONE (0-1451520; 1451520 rows)
Determining min and max w & theoretical beam size... DONE (w=[0.000172088:5470.03] lambdas, maxuvw=25676.3 lambda)
Theoretic beam = 8.03''
Minimal inversion size: 216 x 216, using optimal: 216 x 216
The theoretically suggested number of w-layers (1) i

[2023-05-26 10:33:01,173 | INFO | source.imaging] Finished `wsclean` on `/net/sinatra/vault2-tina/welman/meerkat-src-100/gains/true/true-gains.npz` with 24.5s taken


Fourier transforms...
Gridded visibility count: 1.45152e+06
 == Cleaning (1) ==
Estimated standard deviation of background noise: 6.02 mJy
Initial peak: 255.12 mJy at 142,50
Performed 0 iterations in total, 0 in this major iteration with sub-minor optimization.
Deconvolution aborted.
Writing model image...
1 major iterations were performed.
Rendering sources to restored image (beam=8.61''-10.76'', PA=158.38 deg)... DONE
Writing restored image... DONE
Inversion: 00:00:23.748538, prediction: 00:00:00, deconvolution: 00:00:00.008037
